In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
gwas_c_data_dir <- '/scratch/groups/mrivas/public_data/gwas_catalog_20200216' 
gwas_c_file  <- file.path(gwas_c_data_dir, 'gwas_catalog_v1.0.2-associations_e98_r2020-02-08.tsv.gz')


In [3]:
res_dir <- '/oak/stanford/groups/mrivas/projects/biomarkers/phewas/v2_imp_nc'
phewas_hits_ld <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.ld.tsv')
phewas_hits_ld_gwas_catalog <- file.path(res_dir, 'ukb24983_imp_v3.nc.phewas.hits.ld.gwascatalog.tsv')


In [13]:
gwas_c_df <- fread(cmd=paste('zcat', gwas_c_file), sep='\t')


In [29]:
df <- fread(phewas_hits_ld) %>% rename('CHROM' = '#CHROM')

In [24]:
gwas_c_hits <- gwas_c_df %>% 
select(SNPS, 'P-VALUE', 'MAPPED_TRAIT', 'MAPPED_TRAIT_URI') %>% 
filter(SNPS %in% (df %>% select(LD_rsID) %>% unique() %>% pull())) %>%
rename('EBI_P' = 'P-VALUE', 'EBI_TRAIT' = 'MAPPED_TRAIT', 'EBI_URI' = 'MAPPED_TRAIT_URI')


In [25]:
gwas_c_hits %>% dim()

[1] 840   4

In [27]:
gwas_c_hits %>% filter(is.na('EBI_P')) %>% head()

SNPS,EBI_P,EBI_TRAIT,EBI_URI
<chr>,<chr>,<chr>,<chr>


In [44]:
idx_df <- df %>%
arrange(CHROM, POS, GBE_ID) %>%
select(ID) %>%
unique()%>%
mutate(phewas_idx = 1:n())


In [45]:
idx_df %>% dim()

[1] 175   2

In [47]:
df_with_ebi <- df %>% 
left_join(
    idx_df, by=c('ID')
) %>%
full_join(gwas_c_hits, by=c('LD_rsID' = 'SNPS')) %>%
drop_na(LD_ID) %>%
arrange(phewas_idx, CHROM, POS, LD_CHROM, LD_POS)


In [48]:
df_with_ebi %>% 
rename('#CHROM' = 'CHROM') %>%
fwrite(phewas_hits_ld_gwas_catalog, sep='\t')
